In [1]:
import warnings
from importlib import reload

warnings.filterwarnings('ignore')

In [50]:
from clients.gpt_database_logger import GPTDatabaseLogger
import re
import json

GPT_DB_LOGGER_PATH = 'sqlite:///prompt_versions.db'

db_logger = GPTDatabaseLogger(GPT_DB_LOGGER_PATH)

AGENT_API_URL = 'http://localhost:8013'
AGENT_API_URL = 'https://neon-dev.us'

In [11]:
import sys

sys.path.append('/Users/valuamba/projs/components_agent_sales/app/utils')

from html_messages_parser import get_element_messages, get_messages_from_html_file  


In [44]:
from typing import List

def get_conversation_html(file_name: str):
    with open(f'./deals_html/discount_v3/{file_name}.html', 'r') as f:
        return f.read()

def parse_file_name(file_name: str):
    deal_id, message_id = file_name.split('_', 1)
    return deal_id, message_id

def get_conversation_messages(file_name: str) -> List[str]:
    return get_messages_from_html_file(f'./deals_html/discount_v3/{file_name}.html')

def messages_to_pretty_format(messages: List[str]):
    for idx, msg in enumerate(messages):
        print(f'Message: {len(messages) - idx}')
        print(f'```\n' + msg + '\n```\n\n')

def select_json_block(text: str):
    match = re.search(r"```json\n([\s\S]*?)\n```", text)
    if match:
        json_data = match.group(1)
    else:
        raise ValueError("No valid JSON data found in the string.")

    return json.loads(json_data)


def prepare_conversation_to_prompt(messages_from_latest):
    messages_str = ''
    for idx, msg in enumerate(messages_from_latest):
        messages_str += f'Message: {len(messages_from_latest) - idx}\n'
        if 'Offer-Nr.' in msg:
            messages_str += '<Offer from CRM>'
        messages_str += f'```\n' + msg + '\n```\n\n'
    return messages_str

def filter_messages(messages_from_latest, skip: int, lenght: int):
    return messages_from_latest[::-1][skip:skip+lenght][::-1]
    

In [7]:
file_name = '410158_CAFidWpS2O2JN6fPtwQ0SiqdsSG+y9qDr0xEWeZj_c==rEhG62g@mail.gmail.com'


In [8]:
conversation_html = get_conversation_html(file_name)

In [9]:
deal_id, message_id = parse_file_name(file_name)

In [25]:
messages = get_conversation_messages(file_name)

In [26]:
messages_from_start = messages[::-1]

In [ ]:
messages_to_pretty_format(messages_from_start[8:8+3][::-1])

In [47]:
def discount_processing(conversation: str) -> str:
    response = db_logger.create_completion(temperature=0.1, tags='discount_state', model='gpt-4', messages=[
        { "role": "user", "content": f"""
Analyze the messages and try to figure out discount state at conversation.

By 'discount,' I also mean a request to lower the price, etc. The following or other terms may be used for this purpose: - discount, payment terms, lower price, 
your price is too high, provide us a cheaper price, better price, expensive.

The possible discount states: NotHandled, Rejected, RepeatedRequest, NotAsked.

Conversation:
{conversation}

Put the response into ```json``` format like this:
```json
{{
    "state": <discount state>,
    "justification": <justification why client ask for discount> // fill this field only if it specified at customer messages
    "messages_ids": array[] // messages ids where customer or sales manager discuss or mention discount
}}
```
"""}])
    discount_state = select_json_block(response)
    return discount_state
    

In [51]:
prepared_messages = filter_messages(messages, 8, 3)
prepared_conversation = prepare_conversation_to_prompt(prepared_messages)


In [53]:
response = discount_processing(prepared_conversation)

```json
{
    "state": "NotHandled",
    "justification": "The customer has already made the payment and is asking for a discount.",
    "messages_ids": [3]
}
```

--------------------

Note saved without feedback. ID: 4eaa8671-940c-4245-972d-72ad0f3a9756
Input tokens: 861 Output tokens: 42 Total price: 0.03$




Textarea(value='', description='Feedback:', layout=Layout(height='80px', width='70%'), placeholder='Type your …

Button(button_style='success', description='Submit Feedback', style=ButtonStyle(), tooltip='Click to submit fe…

In [ ]:
print(prepared_conversation)

In [63]:
print(prepared_messages[-response['messages_ids'][0]])

good afternoon,
                                                  please could you give
                                                  me a discount for
                                                  already made the
                                                  payment and can you
                                                  tell me how long it
                                                  takes for me to
                                                  receive it in
                                                  Mozambique?<kg1@famaga.de> escreveu em qui.,
                                                  15/06/2023 às 14:44 :


In [99]:
import requests
from collections import defaultdict
from datetime import datetime

class FamagaClient:
    def __init__(self, api_key, session_id):
        self.base_url = "https://api.famaga.org"
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Cookie": f"PHPSESSID={session_id}"
        }

    def list_offers_by_deal_id(self, deal_id):
        """List offers by deal ID"""
        response = requests.get(f"{self.base_url}/requisition?request={deal_id}", headers=self.headers)
        return response.json()

    def parts_selected_at_offer(self, offer_id):
        """Fetch parts selected at a specific offer"""
        response = requests.get(f"{self.base_url}/requisition/{offer_id}/products", headers=self.headers)
        return response.json()

    def list_offers_by_client_id(self, client_id):
        """Retrieve offers by client ID"""
        response = requests.get(f"{self.base_url}/requisition?client={client_id}", headers=self.headers)
        return response.json()

    def list_current_offer_details(self, offer_id):
        """List details of the current offer in a specific format"""
        parts = self.parts_selected_at_offer(offer_id)
        formatted_parts = []
        for part in parts:
            formatted_part = f"({part['articul']} {part['brand_title']}) margin: {part['margin_ru']}%, sell: {part['price_end']}$ qty. {part['count']}"
            formatted_parts.append(formatted_part)
        return "\n".join(formatted_parts)

    def find_parts_client_bought_before(self, client_id, current_offer_parts):
        """Return the parts that a client has bought before"""
        all_offers = self.list_offers_by_client_id(client_id)
        past_parts = []
        for offer in all_offers.get('content', []):
            parts = self.parts_selected_at_offer(offer['request']['id'])
            past_parts.extend(parts)
        
        bought_before = []
        current_parts_articul = [part['articul'] for part in current_offer_parts]
        for part in past_parts:
            if part['articul'] in current_parts_articul:
                bought_before.append(part)
        
        return bought_before

    def get_client_purchase_history_formatted(self, client_id, exluded_deal: int=None):
        """Generate formatted purchase history for a client"""
        history_data = self.list_offers_by_client_id(client_id)
        purchases = defaultdict(list)
        
        for item in history_data.get('content', []):
            date_key = datetime.strptime(item['created'], '%Y-%m-%dT%H:%M:%S%z').date()
            request_id = int(item['request']['id'])
            if exluded_deal is None or request_id != exluded_deal:
                parts = self.parts_selected_at_offer(request_id)
    
                for part in parts:
                    line = f"{item['id']} ({part['articul']} {part['brand_title']}) margin: {part['margin_ru']}%, sell: {part['price_end']}$ qty. {part['count']}, request id {part['request_id']}"
                    purchases[date_key].append(line)
        
        # Formatting the result
        result = "[CLIENT PURCHASE HISTORY]\n**Client purchase history:**\n"
        for date in sorted(purchases):
            result += f"Date: {date}\n"
            result += "\n".join(purchases[date]) + "\n\n"
        result += "[/CLIENT PURCHASE HISTORY]"
        return result




In [100]:
api_key = "YXBpZmFtYWdhcnU6RHpJVFd1Lk1COUV4LjNmdERsZ01YYlcvb0VFcW9NLw"
session_id = "085qpt4eflu39a0dg7hjhr5mdu"
client = FamagaClient(api_key, session_id)

In [68]:
offer_info = client.list_offers_by_deal_id(deal_id)
offer_id = offer_info['content'][0]['request']['id']
client_id = offer_info['content'][0]['request']['firm']['id']

In [102]:
print(client.get_client_purchase_history_formatted(client_id, int(deal_id)))

[CLIENT PURCHASE HISTORY]
**Client purchase history:**
[/CLIENT PURCHASE HISTORY]


In [78]:
deal_id

'410158'